In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage.transform import resize
import os
import tensorflow as tf

from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [ ]:
def load_data(idx,batch_size):
    x = X_train[idx*batch_size:(idx+1)*batch_size]
    X_train_array = []
    for i in x:
        file_name = i
        img = plt.imread(file_name,cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = resize(img, (800, 800, 1))
            img = np.asarray(img)
            X_train_array.append(img)
    y = Y_train[idx*batch_size:(idx+1)*batch_size]
    x, y = np.array(X_train_array), np.array(y)
    return (x,y)

def batch_generator(batch_size,steps):
    idx = 1
    while True:
        yield load_data(idx-1,batch_size)
        if idx < steps:
            idx += 1
        else:
            idx = 1

def load_two_dataset(path_one, path_two):
  X_dataset_one = []
  X_dataset_two = []
  fil_array = os.listdir(path_one)
  for fil in fil_array:
    X_dataset_one.append(path_one + fil)
  fil_array = os.listdir(path_two)
  for fil in fil_array:
    X_dataset_two.append(path_two+fil)
  one = 0
  two = 0
  X_dataset = []
  Y_dataset = []
  while one + two < len(X_dataset_one) + len(X_dataset_two):
    if one < len(X_dataset_one):
      X_dataset.append(X_dataset_one[one])
      Y_dataset.append(0)
      one += 1
    if two < len(X_dataset_two):
      X_dataset.append(X_dataset_two[two])
      Y_dataset.append(1)
      two += 1
  return (X_dataset, Y_dataset)

def load_dataset(path_one, path_two, path_three):
  X_dataset_one = []
  X_dataset_two = []
  X_dataset_three = []
  fil_array = os.listdir(path_one)
  for fil in fil_array:
    X_dataset_one.append(path_one + fil)
  fil_array = os.listdir(path_two)
  for fil in fil_array:
    X_dataset_two.append(path_two+fil)
  fil_array = os.listdir(path_three)
  for fil in fil_array:
    X_dataset_three.append(path_three+fil)
  X_dataset = []
  Y_dataset = []
  one = 0
  two = 0
  three = 0
  while one + two + three<len(X_dataset_one)+len(X_dataset_two) + len(X_dataset_three):
    if one < len(X_dataset_one):
      X_dataset.append(X_dataset_one[one])
      Y_dataset.append([1,0,0])
      one += 1
    if two < len(X_dataset_two):
      X_dataset.append(X_dataset_two[two])
      Y_dataset.append([0,1,0])
      two += 1
    if three < len(X_dataset_three):
      X_dataset.append(X_dataset_three[three])
      Y_dataset.append([0,0,1])
      three += 1
  return X_dataset, Y_dataset

def load_Test_dataset(X_dataset_paths):
  X_test = []
  for file in X_dataset_paths:
    img = plt.imread(file,cv2.IMREAD_GRAYSCALE)
    if img is not None:
      img = resize(img,(800,800,1))
      img = np.asarray(img)
      X_test.append(img)
  return X_test

def model_tree(example):
  exm = resize(example,(1,800,800,1))
  P_global = model_global.predict(exm)
  if P_global[0][0] > P_global[0][1] and P_global[0][0] > P_global[0][2]:
    P_OCT_one = model_OCT_one.predict(exm)
    if P_OCT_one[0][0] > P_OCT_one[0][1]:
      return 10
    else:
      P_OCT_two = model_OCT_two.predict(exm)
      if P_OCT_two[0][0] > P_OCT_two[0][1] and P_OCT_two[0][0] > P_OCT_two[0][2]:
        return 111
      elif P_OCT_two[0][1] > P_OCT_two[0][0] and P_OCT_two[0][1] > P_OCT_two[0][2]:
        return 112
      else:
        return 113
  elif P_global[0][1] > P_global[0][0] and P_global[0][1] > P_global[0][2]:
    P_MRI_one = model_MRI_one.predict(exm)
    if P_MRI_one[0][0] > P_MRI_one[0][1]:
      return 20
    else:
      P_MRI_two = model_MRI_two.predict(exm)
      if P_MRI_two[0][0] > P_MRI_two[0][1] and P_MRI_two[0][0] > P_MRI_two[0][2]:
        return 211
      elif P_MRI_two[0][1] > P_MRI_two[0][0] and P_MRI_two[0][1] > P_MRI_two[0][2]:
        return 212
      else:
        return 213
  else:
    P_XRAY_one = model_XRAY_one.predict(exm)
    if P_XRAY_one[0][0] > P_XRAY_one[0][1]:
      return 30
    else:
      P_XRAY_two = model_XRAY_two.predict(exm)
      if P_XRAY_two[0][0] > P_XRAY_two[0][1]:
        return 311
      else:
        return 322


In [ ]:
!unzip /content/drive/MyDrive/Global_SNS_train.zip

In [ ]:
X_train, Y_train = load_dataset('/content/Global_SNS_train/OCT2017/', '/content/Global_SNS_train/MRI/', '/content/Global_SNS_train/Chest/')
X_test = np.array(load_Test_dataset(X_train[:143]))
Y_test = np.array(Y_train[:143])
X_train = X_train[143:]
Y_train = Y_train[143:]

In [ ]:
nd_epochs = 3
batch_size = 5
size_BD = len(X_train)
steps_per_epochs = np.ceil(size_BD/batch_size)
my_train_generator = batch_generator(batch_size,steps_per_epochs)


In [ ]:
model_global = Sequential()

model_global.add(Conv2D(8,kernel_size=3,activation="relu",input_shape = (800,800,1)))
model_global.add(MaxPooling2D(pool_size=(2,2)))

model_global.add(Conv2D(16,kernel_size=3,activation="relu"))
model_global.add(MaxPooling2D(pool_size=(2,2)))

model_global.add(Conv2D(32,kernel_size=3,activation="relu"))
model_global.add(MaxPooling2D(pool_size=(2,2)))

model_global.add(Conv2D(64,kernel_size=3,activation="relu"))
model_global.add(MaxPooling2D(pool_size=(2,2)))

model_global.add(Conv2D(128,kernel_size=3,activation="relu"))
model_global.add(MaxPooling2D(pool_size=(2,2)))

model_global.add(Conv2D(256,kernel_size=3,activation="relu"))
model_global.add(MaxPooling2D(pool_size=(2,2)))

model_global.add(Flatten())
model_global.add(Dense(3,activation="softmax"))

In [ ]:
model_global.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=['accuracy'])
model_global.fit_generator(my_train_generator,steps_per_epoch=steps_per_epochs,epochs=nd_epochs,validation_data=(X_test,Y_test), use_multiprocessing=True)

In [ ]:
X_train.clear()
Y_train.clear()
X_test = []
Y_test = []

len_test = 100
X_train, Y_train = load_two_dataset('/content/SNS_OCT_one/NORMAL/', '/content/SNS_OCT_one/NO/')
X_test = np.array(load_Test_dataset(X_train[:len_test]))
Y_test = np.array(Y_train[:len_test])
X_train = X_train[len_test:]
Y_train = Y_train[len_test:]

In [ ]:
nd_epochs = 5
batch_size = 5
size_BD = len(X_train)
steps_per_epochs = np.ceil(size_BD/batch_size)
my_train_generator = batch_generator(batch_size,steps_per_epochs)

In [ ]:
model_OCT_one = Sequential()

model_OCT_one.add(Conv2D(8,kernel_size=3,activation="relu",input_shape = (800,800,1)))
model_OCT_one.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_one.add(Conv2D(16,kernel_size=3,activation="relu"))
model_OCT_one.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_one.add(Conv2D(32,kernel_size=3,activation="relu"))
model_OCT_one.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_one.add(Conv2D(64,kernel_size=3,activation="relu"))
model_OCT_one.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_one.add(Conv2D(128,kernel_size=3,activation="relu"))
model_OCT_one.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_one.add(Conv2D(256,kernel_size=3,activation="relu"))
model_OCT_one.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_one.add(Conv2D(512,kernel_size=3,activation="relu"))
model_OCT_one.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_one.add(Flatten())
model_OCT_one.add(Dense(2,activation="sigmoid"))

In [ ]:
model_OCT_one.compile(optimizer='adamax', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_OCT_one.fit_generator(my_train_generator,steps_per_epoch=steps_per_epochs,epochs=nd_epochs,validation_data=(X_test,Y_test), use_multiprocessing=True)

In [ ]:
X_train.clear()
Y_train.clear()
X_test = []
Y_test = []

In [ ]:
!unzip /content/drive/MyDrive/SNS_MRI_one.zip

In [ ]:
len_test = 100
X_Normal = []
X_no = []
path_Normal = "/content/SNS_MRI_one/NORMAL/"
path_No = '/content/SNS_MRI_one/NO/'
fil_array = os.listdir(path_Normal)
for fil in fil_array:
  X_Normal.append(path_Normal + fil)
fil_array = os.listdir(path_No)
for fil in fil_array:
  X_no.append(path_No + fil)
X_train = []
Y_train = []
normal = 0
Not = 0
for i in range(len(X_Normal) + len(X_no)):
  if i % 6 == 0 and normal < len(X_Normal):
    X_train.append(X_Normal[normal])
    Y_train.append(0)
    normal += 1
  else:
    X_train.append(X_no[Not])
    Y_train.append(1)
    Not += 1
X_test = np.array(load_Test_dataset(X_train[:len_test]))
Y_test = np.array(Y_train[:len_test])
X_train = X_train[len_test:]
Y_train = Y_train[len_test:]

In [ ]:
nd_epochs = 8
batch_size = 5
size_BD = len(X_train)
steps_per_epochs = np.ceil(size_BD/batch_size)
my_train_generator = batch_generator(batch_size,steps_per_epochs)

In [ ]:
model_MRI_one = Sequential()

model_MRI_one.add(Conv2D(8,kernel_size=3,activation="relu",input_shape = (800,800,1)))
model_MRI_one.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_one.add(Conv2D(16,kernel_size=3,activation="relu"))
model_MRI_one.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_one.add(Conv2D(32,kernel_size=3,activation="relu"))
model_MRI_one.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_one.add(Conv2D(64,kernel_size=3,activation="relu"))
model_MRI_one.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_one.add(Conv2D(128,kernel_size=3,activation="relu"))
model_MRI_one.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_one.add(Conv2D(256,kernel_size=3,activation="relu"))
model_MRI_one.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_one.add(Conv2D(512,kernel_size=3,activation="relu"))
model_MRI_one.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_one.add(Conv2D(1024,kernel_size=3,activation="relu"))
model_MRI_one.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_one.add(Flatten())
model_MRI_one.add(Dense(2,activation="sigmoid"))

In [ ]:
model_MRI_one.compile(optimizer='adamax', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_MRI_one.fit_generator(my_train_generator,steps_per_epoch=steps_per_epochs,epochs=nd_epochs,validation_data=(X_test,Y_test), use_multiprocessing=True)

In [ ]:
X_no.clear()
X_Normal.clear()
X_train.clear()
Y_train.clear()
X_test = []
Y_test = []
fil_array.clear()

In [ ]:
!unzip /content/drive/MyDrive/SNS_XRAY_one.zip

In [ ]:
len_test = 100
X_train, Y_train = load_two_dataset('/content/SNS_XRAY_one/NORMAL/', '/content/SNS_XRAY_one/NO/')
X_test = np.array(load_Test_dataset(X_train[:len_test]))
Y_test = np.array(Y_train[:len_test])
X_train = X_train[len_test:]
Y_train = Y_train[len_test:]

In [ ]:
nd_epochs = 4
batch_size = 5
size_BD = len(X_train)
steps_per_epochs = np.ceil(size_BD/batch_size)
my_train_generator = batch_generator(batch_size,steps_per_epochs)

In [ ]:
model_XRAY_one = Sequential()

model_XRAY_one.add(Conv2D(8,kernel_size=3,activation="relu",input_shape = (800,800,1)))
model_XRAY_one.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_one.add(Conv2D(16,kernel_size=3,activation="relu"))
model_XRAY_one.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_one.add(Conv2D(32,kernel_size=3,activation="relu"))
model_XRAY_one.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_one.add(Conv2D(64,kernel_size=3,activation="relu"))
model_XRAY_one.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_one.add(Conv2D(128,kernel_size=3,activation="relu"))
model_XRAY_one.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_one.add(Conv2D(256,kernel_size=3,activation="relu"))
model_XRAY_one.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_one.add(Conv2D(512,kernel_size=3,activation="relu"))
model_XRAY_one.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_one.add(Flatten())
model_XRAY_one.add(Dense(2,activation="sigmoid"))

In [ ]:
model_XRAY_one.compile(optimizer='adamax', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_XRAY_one.fit_generator(my_train_generator,steps_per_epoch=steps_per_epochs,epochs=nd_epochs,validation_data=(X_test,Y_test), use_multiprocessing=True)

In [ ]:
X_train.clear()
Y_train.clear()
X_test = []
Y_test = []

In [ ]:
!unzip /content/drive/MyDrive/SNS_OCT_two.zip

In [ ]:
len_test = 100
X_train, Y_train = load_dataset('/content/SNS_OCT_two/CNV/', '/content/SNS_OCT_two/DME/', '/content/SNS_OCT_two/DRUSDEN/')
X_test = np.array(load_Test_dataset(X_train[:len_test]))
Y_test = np.array(Y_train[:len_test])
X_train = X_train[len_test:]
Y_train = Y_train[len_test:]

In [ ]:
nd_epochs = 4
batch_size = 5
size_BD = len(X_train)
steps_per_epochs = np.ceil(size_BD/batch_size)
my_train_generator = batch_generator(batch_size,steps_per_epochs)

In [ ]:
model_OCT_two = Sequential()

model_OCT_two.add(Conv2D(8,kernel_size=3,activation="relu",input_shape = (800,800,1)))
model_OCT_two.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_two.add(Conv2D(16,kernel_size=3,activation="relu"))
model_OCT_two.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_two.add(Conv2D(32,kernel_size=3,activation="relu"))
model_OCT_two.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_two.add(Conv2D(64,kernel_size=3,activation="relu"))
model_OCT_two.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_two.add(Conv2D(128,kernel_size=3,activation="relu"))
model_OCT_two.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_two.add(Conv2D(256,kernel_size=3,activation="relu"))
model_OCT_two.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_two.add(Conv2D(512,kernel_size=3,activation="relu"))
model_OCT_two.add(MaxPooling2D(pool_size=(2,2)))

model_OCT_two.add(Flatten())
model_OCT_two.add(Dense(3,activation="softmax"))

In [ ]:
model_OCT_two.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=['accuracy'])
model_OCT_two.fit_generator(my_train_generator,steps_per_epoch=steps_per_epochs,epochs=nd_epochs,validation_data=(X_test,Y_test), use_multiprocessing=True)

In [ ]:
X_train.clear()
Y_train.clear()
X_test = []
Y_test = []

In [ ]:
!unzip /content/drive/MyDrive/SNS_MRI_two.zip

In [ ]:
len_test = 143
X_train, Y_train = load_dataset('/content/SNS_MRI_two/glioma_tumor/', '/content/SNS_MRI_two/meningioma_tumor/', '/content/SNS_MRI_two/pituitary_tumor/')
X_test = np.array(load_Test_dataset(X_train[:len_test]))
Y_test = np.array(Y_train[:len_test])
X_train = X_train[len_test:]
Y_train = Y_train[len_test:]

In [ ]:
nd_epochs = 5
batch_size = 5
size_BD = len(X_train)
steps_per_epochs = np.ceil(size_BD/batch_size)
my_train_generator = batch_generator(batch_size,steps_per_epochs)

In [ ]:
model_MRI_two = Sequential()

model_MRI_two.add(Conv2D(8,kernel_size=3,activation="relu",input_shape = (800,800,1)))
model_MRI_two.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_two.add(Conv2D(16,kernel_size=3,activation="relu"))
model_MRI_two.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_two.add(Conv2D(32,kernel_size=3,activation="relu"))
model_MRI_two.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_two.add(Conv2D(64,kernel_size=3,activation="relu"))
model_MRI_two.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_two.add(Conv2D(128,kernel_size=3,activation="relu"))
model_MRI_two.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_two.add(Conv2D(256,kernel_size=3,activation="relu"))
model_MRI_two.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_two.add(Conv2D(512,kernel_size=3,activation="relu"))
model_MRI_two.add(MaxPooling2D(pool_size=(2,2)))

model_MRI_two.add(Flatten())
model_MRI_two.add(Dense(3,activation="softmax"))

In [ ]:
model_MRI_two.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=['accuracy'])
model_MRI_two.fit_generator(my_train_generator,steps_per_epoch=steps_per_epochs,epochs=nd_epochs,validation_data=(X_test,Y_test), use_multiprocessing=True)

In [ ]:
X_train.clear()
Y_train.clear()
X_test = []
Y_test = []

In [ ]:
!unzip /content/drive/MyDrive/SNS_XRAY_two.zip

In [ ]:
len_test = 143
X_train = []
Y_train = []
directory = '/content/SNS_XRAY_two/PNEUMONIA/'
fil_array = os.listdir(directory)
for fil in fil_array:
  X_train.append(directory+fil)
  if 'bacteria' in fil:
    Y_train.append(0)
  elif 'virus' in fil:
    Y_train.append(1)
X_test = np.array(load_Test_dataset(X_train[:len_test]))
Y_test = np.array(Y_train[:len_test])
X_train = X_train[len_test:]
Y_train = Y_train[len_test:]

In [ ]:
nd_epochs = 6
batch_size = 5
size_BD = len(X_train)
steps_per_epochs = np.ceil(size_BD/batch_size)
my_train_generator = batch_generator(batch_size,steps_per_epochs)

In [ ]:
model_XRAY_two = Sequential()

model_XRAY_two.add(Conv2D(8,kernel_size=3,activation="relu",input_shape = (800,800,1)))
model_XRAY_two.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_two.add(Conv2D(16,kernel_size=3,activation="relu"))
model_XRAY_two.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_two.add(Conv2D(32,kernel_size=3,activation="relu"))
model_XRAY_two.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_two.add(Conv2D(64,kernel_size=3,activation="relu"))
model_XRAY_two.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_two.add(Conv2D(128,kernel_size=3,activation="relu"))
model_XRAY_two.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_two.add(Conv2D(256,kernel_size=3,activation="relu"))
model_XRAY_two.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_two.add(Conv2D(512,kernel_size=3,activation="relu"))
model_XRAY_two.add(MaxPooling2D(pool_size=(2,2)))

model_XRAY_two.add(Flatten())
model_XRAY_two.add(Dense(2,activation="sigmoid"))

In [ ]:
model_XRAY_two.compile(optimizer='adamax', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_XRAY_two.fit_generator(my_train_generator,steps_per_epoch=steps_per_epochs,epochs=nd_epochs,validation_data=(X_test,Y_test), use_multiprocessing=True)